<a href="https://colab.research.google.com/github/lemarakis/fastmcp/blob/master/REST_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Εγκατάσταση βιβλιοθηκών
!pip install -q google-generativeai requests

In [2]:
#Ρύθμιση Gemini
from google.colab import userdata
import google.generativeai as genai
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-2.5-flash")

In [3]:
#User question (μία πρόταση, 2 intents)
#user_question = ("πόση μνήμη χρησιμοποιεί ο server αυτή τη στιγμή, πόση CPU και πόσος χώρος υπάρχει στο /tmp?")
#user_question = ("πόση ώρα τρέχει ο server?")
#user_question = ("δώσε μου στοιχεία για τον nginx")
#user_question = ("Καλημέρα ! Πως σε λένε?")
user_question = "πόσο ελεύθερο χώρο έχω στο /tmp και πόση RAM καταναλώνω αυτή την στιγμή"
print(user_question)

πόσο ελεύθερο χώρο έχω στο /tmp και πόση RAM καταναλώνω αυτή την στιγμή


In [4]:
#Χειροκίνητο intent parsing
question = user_question.lower()

need_disk = "χώρο" in question or "δίσκο" in question
need_memory = "ram" in question or "μνήμη" in question


In [5]:
#REST API calls
import requests

BASE_URL = "http://195.251.254.31:8001/api"
results = {}

if need_disk:
    r = requests.get(f"{BASE_URL}/disk", params={"path": "/var"})
    results["disk"] = r.json()

if need_memory:
    r = requests.get(f"{BASE_URL}/memory")
    results["memory"] = r.json()

results


{'disk': {'path': '/var',
  'total_gb': 223.18,
  'used_gb': 60.3,
  'free_gb': 151.47},
 'memory': {'total_gb': 62.8,
  'used_gb': 4.47,
  'available_gb': 58.33,
  'percent': 7.1}}

In [6]:
#Μετατροπή JSON σε φυσική απάντηση

prompt = f"""
Ο χρήστης ρώτησε:
"{user_question}"

Δεδομένα συστήματος:
{results}

Δώσε μία σύντομη και φιλική απάντηση στα Ελληνικά.
"""

response = model.generate_content(prompt)
print(response.text)


Γεια σου!

Αυτή τη στιγμή καταναλώνεις **4.47 GB RAM**, που αντιστοιχεί στο **7.1%** της συνολικής σου μνήμης.

Για τον ελεύθερο χώρο στον φάκελο `/tmp`, δεν έχω αυτή τη στιγμή διαθέσιμες πληροφορίες.
